In [1]:
# load
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import log_loss
from tqdm import tqdm
import lightgbm as lgb
from sklearn.model_selection import KFold
import re
tqdm.pandas(desc="my bar!")

pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)
def wlogloss(targets, preds):
    target_cols = ['any', 'epidural', 'subdural', 'subarachnoid', 'intraventricular', 'intraparenchymal']
    res = 0
    for col in target_cols:
        res += log_loss(targets[col], preds[col+'_pred'])
        if col == 'any':
            res += log_loss(targets[col], preds[col+'_pred'])
    res /= 7
    return res
target_cols = ['any', 'epidural', 'subdural', 'subarachnoid', 'intraventricular', 'intraparenchymal']


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
import pickle
with open('test_stackfeats_1_2.pickle', 'rb') as f:
    test_list1 = pickle.load(f)
with open('train_stackfeats_1_2.pickle', 'rb') as f:
    train1 = pickle.load(f)
with open('test_stackfeats_3_4.pickle', 'rb') as f:
    test_list2 = pickle.load(f)
with open('train_stackfeats_3_4.pickle', 'rb') as f:
    train2 = pickle.load(f)
with open('test_stackfeats_5_6.pickle', 'rb') as f:
    test_list3 = pickle.load(f)
with open('train_stackfeats_5_6.pickle', 'rb') as f:
    train3 = pickle.load(f)
with open('test_stackfeats_7_8.pickle', 'rb') as f:
    test_list4 = pickle.load(f)
with open('train_stackfeats_7_8.pickle', 'rb') as f:
    train4 = pickle.load(f)
with open('test_stackfeats_9_10_11.pickle', 'rb') as f:
    test_list5 = pickle.load(f)
with open('train_stackfeats_9_10_11.pickle', 'rb') as f:
    train5 = pickle.load(f)

In [5]:
test_list = test_list1
for i in range(5):
    tmp = test_list[i]
    tmp = tmp.merge(test_list2[i], on='ID', how='left')
    tmp = tmp.merge(test_list3[i], on='ID', how='left')
    tmp = tmp.merge(test_list4[i], on='ID', how='left')
    tmp = tmp.merge(test_list5[i], on='ID', how='left')
    test_list[i] = tmp
train = train1
train = train.merge(train2.drop(target_cols + ['StudyInstanceUID', 'folds'], axis=1), on='ID', how='left')
train = train.merge(train3.drop(target_cols + ['StudyInstanceUID', 'folds'], axis=1), on='ID', how='left')
train = train.merge(train4.drop(target_cols + ['StudyInstanceUID', 'folds'], axis=1), on='ID', how='left')
train = train.merge(train5.drop(target_cols + ['StudyInstanceUID', 'folds'], axis=1), on='ID', how='left')


In [6]:
# import pickle
# with open('test_8models_stackfeats.pickle', 'wb') as f:
#     pickle.dump(test_list, f)
# with open('train_8model_stackfeats.pickle', 'wb') as f:
#     pickle.dump(train, f)
# import pickle
# with open('test_8models_stackfeats.pickle', 'rb') as f:
#     test_list = pickle.load(f)
# with open('train_8model_stackfeats.pickle', 'rb') as f:
#     train = pickle.load(f)
import pickle
with open('test_10models_stackfeats.pickle', 'wb') as f:
    pickle.dump(test_list, f)
with open('train_10model_stackfeats.pickle', 'wb') as f:
    pickle.dump(train, f)
import pickle
with open('test_10models_stackfeats.pickle', 'rb') as f:
    test_list = pickle.load(f)
with open('train_10model_stackfeats.pickle', 'rb') as f:
    train = pickle.load(f)


In [7]:
df_list = []
for n_fold in range(5):
    appian_feats = pd.read_pickle(f'../prediction/appian_feats/fold{n_fold}_ep2_valid_tta1_feats.pkl')
    df = pd.DataFrame(appian_feats[0]['outputs'], columns=[f'appian_cnn_{i}' for i in range(2048)])
    df['ID'] = appian_feats[0]['ids']
    df_list.append(df)
df_appian_tr = pd.concat(df_list)
train = train.merge(df_appian_tr, on='ID', how='left')

In [8]:
for n_fold in range(5):
    appian_feats = pd.read_pickle(f'../prediction/appian_feats/fold{n_fold}_ep2_test_tta1_feats.pkl')
    df = pd.DataFrame(appian_feats[0]['outputs'], columns=[f'appian_cnn_{i}' for i in range(2048)])
    df['ID'] = appian_feats[0]['ids']
    test_list[n_fold] = test_list[n_fold].merge(df, on='ID', how='left')

In [9]:
import pickle
# with open('test_8models_stackfeats_add_appian_feats.pickle', 'wb') as f:
#     pickle.dump(test_list, f, protocol=4)
# with open('train_8model_stackfeats_add_appian_feats.pickle', 'wb') as f:
#     pickle.dump(train, f, protocol=4)
# import pickle
# with open('test_8models_stackfeats_add_appian_feats.pickle', 'rb') as f:
#     test_list = pickle.load(f)
# with open('train_8model_stackfeats_add_appian_feats.pickle', 'rb') as f:
#     train = pickle.load(f)
with open('test_10models_stackfeats_add_appian_feats.pickle', 'wb') as f:
    pickle.dump(test_list, f, protocol=4)
with open('train_10model_stackfeats_add_appian_feats.pickle', 'wb') as f:
    pickle.dump(train, f, protocol=4)
# import pickle
# with open('test_10models_stackfeats_add_appian_feats.pickle', 'rb') as f:
#     test_list = pickle.load(f)
# with open('train_10model_stackfeats_add_appian_feats.pickle', 'rb') as f:
#     train = pickle.load(f)


# stacking lgbm

In [101]:
# X_cols = train.columns.drop(['ID', 'folds', 'StudyInstanceUID'] + target_cols)
X_cols = test_list[0].columns.drop(['ID'])
len(X_cols)

3488

In [102]:
import gc
gc.collect()

40

In [ ]:
%%time
from sklearn.linear_model import Ridge, LogisticRegression
stack_preds = []
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": 0.1,
          "num_leaves": 5,
           "max_bin": 256,
          "feature_fraction": 0.8,
          "verbosity": 0,
          "min_child_samples": 10,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "subsample": 0.9
          }
res = []
pred_test = []
fi = {}
for c in target_cols:
    fi[c] = []
stack_preds_valid = []
for i in range(5):
    tr = train.query('folds != @i')
    va = train.query('folds == @i')
    preds = pd.DataFrame(np.zeros([len(va), 6]), columns = [c + '_pred' for c in target_cols])
    for tar_col in target_cols:
        tr_D = lgb.Dataset(tr[X_cols], tr[tar_col])
        va_D = lgb.Dataset(va[X_cols], va[tar_col])
        clf = lgb.train(params, tr_D, 10000, valid_sets=va_D, verbose_eval=100,
                                    early_stopping_rounds=120)
        preds[tar_col + '_pred'] = clf.predict(va[X_cols])
        pred_test.append(clf.predict(test_list[i][X_cols]))
        df_fi = pd.DataFrame(clf.feature_importance(importance_type='gain'), index = X_cols, columns=['FI_score'])
        fi[tar_col].append(df_fi)
        print(log_loss(va[tar_col], preds[tar_col + '_pred']))
        res.append(log_loss(va[tar_col], preds[tar_col + '_pred']))
        print('-'*80)
    print('='*80)
    stack_preds.append([va['ID'], preds])

Training until validation scores don't improve for 120 rounds
[100]	valid_0's binary_logloss: 0.101205
[200]	valid_0's binary_logloss: 0.101415
Early stopping, best iteration is:
[96]	valid_0's binary_logloss: 0.101152
0.10115229136373897
--------------------------------------------------------------------------------
Training until validation scores don't improve for 120 rounds
[100]	valid_0's binary_logloss: 0.015441
[200]	valid_0's binary_logloss: 0.0159326
Early stopping, best iteration is:
[85]	valid_0's binary_logloss: 0.0153426
0.015342644771518933
--------------------------------------------------------------------------------
Training until validation scores don't improve for 120 rounds
[100]	valid_0's binary_logloss: 0.0860191
[200]	valid_0's binary_logloss: 0.0870095
Early stopping, best iteration is:
[85]	valid_0's binary_logloss: 0.0858931
0.08589308272054638
--------------------------------------------------------------------------------
Training until validation scores d

In [ ]:
def get_score(res):
    score = 0
    for i, r in enumerate(res):
        if i %6 == 0:
            score += 2*r
        else:
            score += r
    return score/5/7

In [ ]:
get_score(res)

In [ ]:
sub = pd.read_pickle(f'{path}/fold{n_fold}_ep2_test_tta5.pkl')
preds_test_df = pd.DataFrame()
for n_fold in range(5):
    for n_target in range(6):
        preds_test_df[target_cols[n_target] + '_' + str(n_fold) + 'fold'] = pred_test[n_fold*6 + n_target]
preds_test_df.index = test['ID'].values
preds_test_df = preds_test_df.loc[sub[0]['ids']]
pred_test_sorted_list = []
for i in range(30):
    pred_test_sorted_list.append(preds_test_df.iloc[:, i].values)
pred_test_df_list = []
for n_fold in range(5):
    tmp = np.zeros([len(test), 6])
    for n_tar in range(6):
        tmp[:, n_tar] = pred_test_sorted_list[6*n_fold + n_tar]
    pred_test_df = pd.DataFrame(tmp, columns = target_cols)
    pred_test_df_list.append(pred_test_df)
sub = pd.read_pickle(f'{path}/fold{n_fold}_ep2_test_tta5.pkl')
sub_original = pd.read_pickle(f'{path}/fold{n_fold}_ep2_test_tta5.pkl')
for i in range(5):
    sub[i]['outputs'] = pred_test_df_list[i][target_cols].values
for i in range(5):
    for ii in range(6):
        print(np.corrcoef(sub[i]['outputs'][:, ii], sub_original[i]['outputs'][:, ii])[0, 1])
path_to = '../stack_prediction/pred_test_1103_add_appian_feats.pkl'
with open(path_to, 'wb') as f:
    pickle.dump(sub, f)
path_to = f'../stack_prediction/pred_valid_1103_add_appian_feats.pkl'
with open(path_to, 'wb') as f:
    pickle.dump(stack_preds, f)

# check the correlation of submits

In [12]:
len(appian_feats)

1

In [2]:
a1 = pd.read_csv('../model/model001/stack_1031_nobag.csv')
a2 = pd.read_csv('../model/seresnext410/stack_1031_resnext410.csv')
np.corrcoef(a1['Label'], a2['Label'])

array([[1.        , 0.99604635],
       [0.99604635, 1.        ]])

In [3]:
a1 = pd.read_csv('../model/seresnext410/stack_1031_resnext410_dart.csv')
a2 = pd.read_csv('../model/seresnext410/stack_1031_resnext410.csv')
np.corrcoef(a1['Label'], a2['Label'])

array([[1.        , 0.99912219],
       [0.99912219, 1.        ]])

In [4]:
a1 = pd.read_csv('../model/model001/stack_1031_nobag.csv')
a2 = pd.read_csv('../model/inceptionv4/stack_1031_inceptionv4.csv')
np.corrcoef(a1['Label'], a2['Label'])

array([[1.        , 0.99264155],
       [0.99264155, 1.        ]])

In [9]:
p1 = pd.read_csv('../model/inceptionv4/stack_1031_inceptionv4.csv')
p2 = pd.read_csv('../model/seresnext410/stack_1031_resnext410.csv')
p3 = pd.read_csv('../model/seresnext410/stack_1031_resnext410_dart.csv')
p4 = pd.read_csv('../model/model001/stack_1031_2.csv')
p5 = pd.read_csv('../model/model001/stack_1031_model001_dart.csv')
p6 = pd.read_csv('../model/inceptionv4/stack_1031_inceptionv4_dart.csv')
p = p1['Label'] * 0.15 + p5['Label'] * 0.15 + p2['Label'] * 0.15 + p3['Label'] * 0.15 + p4['Label'] * 0.2 + p6['Label'] * 0.2
a1['Label'] = p.values * 0.98
a1.to_csv('1031_stack_seresnext512_30_seresnext410_25_seresnext410dart_25_seresnext410_20_2_multi098.csv', index=False)

0.00010550126945637375

In [102]:
a1 = pd.read_csv('../model/model001/stack_1031_nobag.csv')
a2 = pd.read_csv('../model/model001/stack_1031_2.csv')
np.corrcoef(a1['Label'], a2['Label'])

array([[1.        , 0.99931158],
       [0.99931158, 1.        ]])